# Subjective Evaluation (ver. 2023.03.01)

In [56]:
import os, glob, mido, pickle, shutil, random
import muspy as mu
import os, glob, mido
import numpy as np
from mido import MidiFile
import pretty_midi as pr
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import seaborn as sns
from tqdm import tqdm
from math import log10,floor
from pathlib import Path
import pandas as pd

## Sample Selection & Processing

### Duration Filtering: 10-60s Outputs

In [12]:
directory='/data/data1/users/astais/Human-Evaluation/Midi Samples/'
# Count number of midis in directory
mid_num=len([f for f in Path(directory).rglob('*.mid*')])

count=0
# iterate over files in directory
for f in tqdm(Path(directory).rglob('*.mid*'),total=mid_num,
              desc="Processing "+str(mid_num)+" midi files"):
        try:
            mid=MidiFile(f)
            if(mid.length<10 or mid.length>60):
                os.remove(str(f))
                count+=1   
        except KeyboardInterrupt:
            print('Keyboard Interrupt.')
            break
        except:
            print("Error happened. File: "+str(f))

print('Done!')
print(count)


Processing 7800 midi files: 100%|███████████| 7800/7800 [02:13<00:00, 58.30it/s]

Done!
1694


### Random Sampling

In [4]:
directory='/data/data1/users/astais/Human-Evaluation/Midi Samples/'
save_dir='/data/data1/users/astais/Human-Evaluation/Midi-Samples-Sampled/'
# Count number of midis in directory

# all directories for sampling
# directories=[directory+'music-transformer/music-transformer_maestro-v3.0.0/',
#              directory+'music-transformer/music-transformer_GiantMIDI-Piano/',
#             directory+'music-transformer/music-transformer_ailabs1k7/',
#             directory+'music-transformer/music-transformer_Rock-Piano-MIDI-Dataset/',
#             directory+'music-transformer/music-transformer_adl-piano-midi/',
#              directory+'music-transformer/music-transformer_Los-Angeles-MIDI-Dataset-segment/',
#              directory+'perceiver-ar/perceiver-ar_maestro-v3.0.0/',
#              directory+'perceiver-ar/perceiver-ar_GiantMIDI-Piano/',
# directories=[
#             directory+'perceiver-ar/perceiver-ar_ailabs1k7/',
#             directory+'perceiver-ar/perceiver-ar_Rock-Piano-MIDI-Dataset/',
#             directory+'perceiver-ar/perceiver-ar_adl-piano-midi/',
#              directory+'perceiver-ar/perceiver-ar_Los-Angeles-MIDI-Dataset-segment/'
#             ]

directories=[directory+'/training-datasets/']
num=96
for Dir in tqdm(directories): 
    print(Dir)
    # count number of midis in directory
    mid_num=len(tqdm([f for f in Path(Dir).rglob('*.mid*')]))
    file_list = list(Path(Dir).glob(f"**/*.mid*"))
    if not len(file_list):
        print("No files found.")

    # iterate over files in directory and randomly copy 
    for i in tqdm(range(num),desc="Choosing "+str(num)+" samples for this directory."):
        while True: 
            try:
                # pick a random file from the file list
                rand = random.randint(0, len(file_list) - 1)
                file=str(file_list[rand])
                
                # test that the file can be processed
                pr.PrettyMIDI(file)
                mu.from_mido(MidiFile(file), duplicate_note_mode='fifo')
                # copy file to primers folder
                shutil.copy(file,save_dir)
            except:            
                # Error happened, continue
                print("Error happened. File: "+file)
                continue
            else:
                break

    print('Done!')


  0%|                                                     | 0/1 [00:00<?, ?it/s]

/data/data1/users/astais/Human-Evaluation/Midi Samples//training-datasets/



  0%|                                                   | 0/450 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.89it/s]

Done!


## Listening Test Page

### Create basic and pro users rating dictionaries

In [ ]:
basic={}
pro={}
directory='/home/astais/survey/static/samples/'
mp3_num=len([f for f in Path(directory).rglob('*.mp3*')])

# iterate over files in directory
for f in tqdm(Path(directory).rglob('*.mp3*'),total=mp3_num,
              desc="Processing "+str(mp3_num)+" mp3 files"):
        try:
            basic[f.name]=[[],[],[],[],[],[],[],[]]
            pro[f.name]=[[],[],[],[],[],[],[],[]]
        except KeyboardInterrupt:
            print('Keyboard Interrupt.')
            break
        except:
            print("Error happened. File: "+str(f))

print('Done!')

with open('/home/astais/survey/static/ratings/basic.pkl', 'wb') as f:
    pickle.dump(basic, f, pickle.HIGHEST_PROTOCOL)
    
with open('/home/astais/survey/static/ratings/pro.pkl', 'wb') as f:
    pickle.dump(pro, f, pickle.HIGHEST_PROTOCOL)
    
with open('/home/astais/survey/static/ratings/basic.pkl', 'rb') as f:
    x=pickle.load(f)
    
with open('/home/astais/survey/static/ratings/pro.pkl', 'rb') as f:
    y=pickle.load(f)
    
print(x)
print(y)


### Read Rating Dictionaries

In [ ]:
files = os.listdir('/home/astais/survey/static/samples/')
print(files)

with open('/home/astais/survey/static/samples-list.pkl', 'wb') as f:
    pickle.dump(files, f)
    
with open('/home/astais/survey/static/samples-list.pkl', 'rb') as f:
    z=pickle.load(f)
    
print(z)
print(files==z)


In [12]:
with open('/home/astais/survey/static/ratings/ratings_pro.pkl', 'rb') as f:
    y=pickle.load(f)
with open('/home/astais/survey/static/ratings/ratings_basic.pkl', 'rb') as f:
    z=pickle.load(f)
print(len(y))
print(len(z))

672
672


## Subjective Metrics Dataframe

In [37]:
import pandas as pd
import pickle

d_n={
    'ratings_basic.pkl':"Basic",
    'ratings_pro.pkl':"Pro",
    "music-transformer": "Music Transformer Outputs",
    "perceiver-ar": "Perceiver-AR Outputs",
    "training-datasets": "Training Dataset",
    "adl-piano-midi": "adl-piano-midi" , 
    "ailabs1k7": "ailabs1k7",
    "ailabs1k17": "ailabs1k7",
    "GiantMIDI-Piano": "GiantMIDI-Piano",
    "Los-Angeles-MIDI-Dataset-segment": "Los-Angeles-MIDI-\nDataset-segment", 
    "maestro-v3.0.0": "maestro-v3.0.0",
    "Rock-Piano-MIDI-Dataset": "Rock-Piano-\nMIDI-Dataset"
}


# create lists to store the values
sample_names = []
dataset_types = []
datasets = []
primer_datasets=[]
users=[]
similar_music = []
emotions = []
melody = []
harmony = []
rhythm = []
genre = []
creator = []
rating = []

for file in ['ratings_basic.pkl','ratings_pro.pkl']:
    
    # load the dictionary
    with open('/home/astais/'+file, 'rb') as f:
        ratings_dict = pickle.load(f)
    
    # iterate over the dictionary and extract the values
    for sample, ratings in ratings_dict.items():
        sample_name = sample.split("_no-primer")[0]
        dataset_type = d_n[sample.split("_")[0]]
        dataset = d_n[sample.split("_")[1]]

        if(dataset_type=="Training Dataset"):
            primer_dataset=dataset
        else:
            primer_dataset = d_n[sample.split("_")[2]]

        # If the sample has no ratings, skip
        if (ratings==[[], [], [], [], [], [], [], []]):
            continue

        for i in range(len(ratings[0])):
            sample_names.append(sample_name)
            dataset_types.append(dataset_type)
            datasets.append(dataset)
            primer_datasets.append(primer_dataset)
            users.append(d_n[file])
            similar_music.append(ratings[0][i])
            emotions.append(ratings[1][i])
            melody.append(ratings[2][i])
            harmony.append(ratings[3][i])
            rhythm.append(ratings[4][i])
            genre.append(ratings[5][i])
            creator.append(ratings[6][i])
            rating.append(ratings[7][i])


# create the DataFrame
df = pd.DataFrame({
    "Sample Name": sample_names,
    "Dataset Type": dataset_types,
    "Dataset": datasets,
    "Primer Dataset": primer_datasets,
    "Musical Knowledge": users,
    "Familiarity": similar_music,
    "Emotion": emotions,
    "Melodiousness": melody,
    "Harmonicity": harmony,
    "Rhythmicity": rhythm,
    "Genre": genre,
    "Naturalness": creator,
    "Rating": rating
})

pd.set_option('display.max_colwidth', None)
print(df)

# Save dataframe to pickle file
df.to_pickle('/home/astais/ratings_dataframe.pkl')

                                                                           Sample Name  \
0       music-transformer_GiantMIDI-Piano_Los-Angeles-MIDI-Dataset-segment_97_20230106   
1                               music-transformer_maestro-v3.0.0_ailabs1k7_71_20230104   
2                               music-transformer_maestro-v3.0.0_ailabs1k7_71_20230104   
3                               music-transformer_adl-piano-midi_ailabs1k7_81_20230106   
4                               music-transformer_adl-piano-midi_ailabs1k7_81_20230106   
...                                                                                ...   
1375  perceiver-ar_Los-Angeles-MIDI-Dataset-segment_GiantMIDI-Piano_40_20221207_161342   
1376                     perceiver-ar_maestro-v3.0.0_adl-piano-midi_22_20221201_032308   
1377                     perceiver-ar_maestro-v3.0.0_adl-piano-midi_22_20221201_032308   
1378                             music-transformer_ailabs1k7_maestro-v3.0.0_4_20230106   
1379      

### Subjective Metrics Statistics

In [53]:
d_n2={
    "adl-piano-midi": "adl-piano-midi" , 
    "ailabs1k7": "ailabs1k7",
    "ailabs1k17": "ailabs1k7",
    "GiantMIDI-Piano": "GiantMIDI-Piano",
    "Los-Angeles-MIDI-\nDataset-segment": "Los-Angeles-MIDI-Dataset-segment", 
    "maestro-v3.0.0": "maestro-v3.0.0",
     "Rock-Piano-\nMIDI-Dataset": "Rock-Piano-MIDI-Dataset"
}

excel_df_basic=pd.DataFrame()
excel_df_pro=pd.DataFrame()

for dataset in datasets:
    df_sub = df[(df["Dataset"] == dataset) & (df["Dataset Type"] == "Training Dataset") & (df["Musical Knowledge"] == "Basic")].describe()
    df_sub2 = df[(df["Dataset"] == dataset) & (df["Dataset Type"] == "Music Transformer Outputs") & (df["Musical Knowledge"] == "Basic")].describe()
    df_sub3 = df[(df["Dataset"] == dataset) & (df["Dataset Type"] == "Perceiver-AR Outputs") & (df["Musical Knowledge"] == "Basic")].describe()
    df_sub4 = df[(df["Dataset"] == dataset) & (df["Dataset Type"] == "Training Dataset") & (df["Musical Knowledge"] == "Pro")].describe()
    df_sub5 = df[(df["Dataset"] == dataset) & (df["Dataset Type"] == "Music Transformer Outputs") & (df["Musical Knowledge"] == "Pro")].describe()
    df_sub6 = df[(df["Dataset"] == dataset) & (df["Dataset Type"] == "Perceiver-AR Outputs") & (df["Musical Knowledge"] == "Pro")].describe()
    
    df_sub.insert(loc=0, column='Dataset Type', value=["Training Dataset"] *  len(df_sub))
    df_sub.insert(loc=1, column='Dataset', value=[d_n2[dataset]] * len(df_sub))
    df_sub2.insert(loc=0, column='Dataset Type', value=["Music Transformer Outputs"] *  len(df_sub2))
    df_sub2.insert(loc=1, column='Dataset', value=[d_n2[dataset]] * len(df_sub2))
    df_sub3.insert(loc=0, column='Dataset Type', value=["Perceiver-AR Outputs"] *  len(df_sub3))
    df_sub3.insert(loc=1, column='Dataset', value=[d_n2[dataset]]* len(df_sub3))
    df_sub4.insert(loc=0, column='Dataset Type', value=["Training Dataset"] *  len(df_sub4))
    df_sub4.insert(loc=1, column='Dataset', value=[d_n2[dataset]] * len(df_sub4))
    df_sub5.insert(loc=0, column='Dataset Type', value=["Music Transformer Outputs"] *  len(df_sub5))    
    df_sub5.insert(loc=1, column='Dataset', value=[d_n2[dataset]] * len(df_sub5))
    df_sub6.insert(loc=0, column='Dataset Type', value=["Perceiver-AR Outputs"] *  len(df_sub6))
    df_sub6.insert(loc=1, column='Dataset', value=[d_n2[dataset]] * len(df_sub6))
    
    df_sub.drop(["Melodiousness","Harmonicity", "Rhythmicity","Genre"], inplace=True, axis=1)
    df_sub2.drop(["Melodiousness","Harmonicity", "Rhythmicity","Genre"], inplace=True, axis=1)
    df_sub3.drop(["Melodiousness","Harmonicity", "Rhythmicity","Genre"], inplace=True, axis=1)
    df_sub4.drop(["Familiarity"], inplace=True, axis=1)
    df_sub5.drop(["Familiarity"], inplace=True, axis=1)
    df_sub6.drop(["Familiarity"], inplace=True, axis=1)

    df_sub.drop(["min","25%", "50%","75%","max"], inplace=True, axis=0)
    df_sub2.drop(["min","25%", "50%","75%","max"], inplace=True, axis=0)
    df_sub3.drop(["min","25%", "50%","75%","max"], inplace=True, axis=0)
    df_sub4.drop(["min","25%", "50%","75%","max"], inplace=True, axis=0)
    df_sub5.drop(["min","25%", "50%","75%","max"], inplace=True, axis=0)
    df_sub6.drop(["min","25%", "50%","75%","max"], inplace=True, axis=0)
    
#     df_sub.to_excel(d_n2[dataset]+"_Training-Dataset_basic.xlsx")
#     df_sub2.to_excel(d_n2[dataset]+"_Music Transformer Outputs_basic.xlsx")
#     df_sub3.to_excel(d_n2[dataset]+"_Perceiver-AR Outputs_basic.xlsx")
#     df_sub4.to_excel(d_n2[dataset]+"_Training-Dataset_pro.xlsx")
#     df_sub5.to_excel(d_n2[dataset]+"_Music Transformer Outputs_pro.xlsx")
#     df_sub6.to_excel(d_n2[dataset]+"_Perceiver-AR Outputs_pro.xlsx")

    excel_df_basic=pd.concat([excel_df_basic,df_sub,df_sub2,df_sub3])
    excel_df_pro=pd.concat([excel_df_pro,df_sub4,df_sub5,df_sub6])

print(excel_df_basic)
print(excel_df_pro)
    
excel_df_basic.to_excel("subjective_metrics_basic.xlsx")
excel_df_pro.to_excel("subjective_metrics_pro.xlsx")

                    Dataset Type                           Dataset  \
count           Training Dataset                    adl-piano-midi   
mean            Training Dataset                    adl-piano-midi   
std             Training Dataset                    adl-piano-midi   
count  Music Transformer Outputs                    adl-piano-midi   
mean   Music Transformer Outputs                    adl-piano-midi   
std    Music Transformer Outputs                    adl-piano-midi   
count       Perceiver-AR Outputs                    adl-piano-midi   
mean        Perceiver-AR Outputs                    adl-piano-midi   
std         Perceiver-AR Outputs                    adl-piano-midi   
count           Training Dataset                         ailabs1k7   
mean            Training Dataset                         ailabs1k7   
std             Training Dataset                         ailabs1k7   
count  Music Transformer Outputs                         ailabs1k7   
mean   Music Transfo

### Subjective Metrics Plots

In [94]:
metrics=[ "Familiarity",
    "Emotion",
    "Melodiousness",
    "Harmonicity",
    "Rhythmicity",
    "Genre",
    "Naturalness",
    "Rating"]

df_pro = df[df["Musical Knowledge"] == "Pro"]
df_basic = df[df["Musical Knowledge"] == "Basic"]
# print(df_pro)
# print(df_basic)

for metric in tqdm(["Emotion", "Melodiousness", "Harmonicity", "Rhythmicity", "Genre", "Naturalness", "Rating"]):    
    
    # Barplot
    sns.set(rc={'figure.figsize':(12,8)})
    g = sns.barplot(data=df_pro, x="Dataset", y=metric, errorbar='sd', capsize=.1, hue='Dataset Type',hue_order=
                    ['Training Dataset','Music Transformer Outputs', 'Perceiver-AR Outputs'])
    if(metric=="Naturalness"):
        g.set(ylim=(1,2))
    else:
        g.set(ylim=(1,5))
    g.set(title =metric+" Barplot")
    g.xaxis.set_minor_locator(AutoMinorLocator(5))
    g.yaxis.set_minor_locator(AutoMinorLocator(5))
    g.grid(which='minor', alpha=0.3)
    g.grid(which='major', alpha=0.6)
    g.figure.savefig(metric+"_barplot_pro.png",dpi=300)
    g.figure.show()
    g.figure.clf()
    
#     # Violin plot
#     sns.set(rc={'figure.figsize':(12,8)})
#     g = sns.violinplot(data=df_pro, x="Dataset", y=metric, hue='Dataset Type',hue_order=
#                     ['Training Dataset','Music Transformer Outputs', 'Perceiver-AR Outputs'])
#     g.set(title =metric+" Violinplot")
#     g.xaxis.set_minor_locator(AutoMinorLocator(5))
#     g.yaxis.set_minor_locator(AutoMinorLocator(5))
#     g.grid(which='minor', alpha=0.3)
#     g.grid(which='major', alpha=0.6)
    
#     g.figure.savefig(metric+"_violinplot.png",dpi=300)
#     g.figure.show()
#     g.figure.clf()

for metric in tqdm(["Familiarity","Emotion", "Naturalness", "Rating"]):    
    
    # Barplot
    sns.set(rc={'figure.figsize':(12,8)})
    g = sns.barplot(data=df_basic, x="Dataset", y=metric, errorbar='sd', capsize=.1, hue='Dataset Type',hue_order=
                    ['Training Dataset','Music Transformer Outputs', 'Perceiver-AR Outputs'])
    if(metric=="Naturalness"):
        g.set(ylim=(1,2))
    else:
        g.set(ylim=(1,5))
    g.set(title =metric+" Barplot")
    g.xaxis.set_minor_locator(AutoMinorLocator(5))
    g.yaxis.set_minor_locator(AutoMinorLocator(5))
    g.grid(which='minor', alpha=0.3)
    g.grid(which='major', alpha=0.6)
    g.figure.savefig(metric+"_barplot_basic.png",dpi=300)
    g.figure.show()
    g.figure.clf()

100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.47it/s]


<Figure size 1200x800 with 0 Axes>